[Download the slides from here.](https://www.dropbox.com/s/ruvi1mcagskglhd/Breakfast_4.pdf?dl=0)

<img src=https://hiverhq.com/blog/wp-content/uploads/2016/01/AB-Testing.jpg width=500>


A/B Testing
-----------

The mechanism behind randomized trials is underneath a host of processes that are optimizing headlines, moving content around on home pages, and testing out what kind of content is pushed out to social media. [Have a read about what Buzzfeed does.](https://ijnet.org/en/blog/adaptation-ab-testing-and-analytics-how-buzzfeed-optimizes-news-its-audience) We are now going to load up the data given to us by the New York Times. [Pull it from Dropbox](https://www.dropbox.com/s/x8ud9taqg12s7c4/nyt.csv?dl=0) and place it in the same folder as your notebook. It's a little old now, but the principles are the same. 

The treatment, you will recall, was replacing a Tab layout versus a List layout for the top cities and most e-mailed stories. Again, this was an old version of the site. The treatment is stored in a variable called "Variation" (which page variation were people shown) and "IfClicked" (did they click or not). Read in the data and then have a look.

In [ ]:
library(dplyr)
library(ggplot2)

In [ ]:
nyt = read.csv("nyt.csv")
head(nyt)

The variables above include the following.

> User_ID - A unique number for each visitor
<br> UserVisit_ID - A unique number for each visit
<br> StartTime_SSE - UNIX time for the start of the visit
<br> StartTime_English - A more humanly readable version of the time
<br> VisitLength - The number of seconds the visitor was reading the Travel Section pages
<br> Variation - The version of the page they received
<br> RefererURL - The page they clicked on to get to the Travel Section (if any)
<br> EntryPageUrl - The first page on nytimes.com they visited
<br> Pageviews - The number of pages viewed in the Travel Section
<br> TotalVisits - The total number of visits to the site
<br> TimeSinceFirstVisit (days) - How long had it been since their first visit
<br> UserAgent - Their browser
<br> TotalClicks - How many times did they ckick on the "most popular" field
<br> IfClicked - 0/1 did they click on the "most popular" field at least once

This data set was collected in 2008 (sadly) and even then you get an idea for how much information sites have to help adapt content. A/B testing is just one strategy. There is plenty here to build up kind of profile of your viewing habits.

Here is how you make a table from just "Variation" and "IfClicked" using the funciton select() that only keeps the named columns from "nyt". So, what does this say?

In [ ]:
table(select(nyt,Variation,IfClicked))

Following how we proceeded with Hill's data, we can make a copy of the data set and then shuffle the treatment column, under the null hypothesis that both Tabs and Lists encourage people to click on the "most popular" field equally. Let's have a look at a few simulated tables and get a sense of how likely, say, having 1200 people who saw Tabs click on the field. Is it rare, suggesting the null hypothesis is wrong; or is it consistent with the null distribution?

In [ ]:
newnyt <- nyt

We had been using `pull()` to extract the data from a column in a table. It is stored in a "vector" which is just an R data structure for storing an ordered collection. Here, the column `Variation` is just a series of values `Tabs` or `Lists`, one corresponding to each visitor. In addition to the `dplyr` command `pull()` we can also extract data using the dollar sign. 

Here we extract the `Variation` column from `nyt`, mix it up (permute it) using `sample()`, which works for vectors, and then stuff it back into `newnyt`. 

In [ ]:
# replace the Variation column with a permutation...
# do this a few times and watch the count of people in the "died"-"penicillin" cell

newnyt$Variation <- sample(nyt$Variation)
table(select(newnyt,Variation,IfClicked))

Because this data set is big (130k rows), we will only simulate 1000 times. The results are clear enough even with this smaller number.

In [ ]:
newnyt$Variation <- sample(nyt$Variation)
newtrials <- summarize(newnyt,tabclick=sum(Variation=="Tabs" & IfClicked==1))

for(i in 1:1000){
    newnyt$Variation <- sample(nyt$Variation)
    newtrials <- bind_rows(newtrials,
                           summarize(newnyt,tabclick=sum(Variation=="Tabs" & IfClicked==1)))
}

Now, let's have a look at our null distribution. Under the hypothesis that tabs and lists entice visitors equally to click on the links they contain, let's conduct our permutation test. 

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

ggplot() + 
    geom_histogram(aes(x=tabclick),newtrials,bins=30,color="white",fill="lightblue") +
    ggtitle("Null Distribution")

For reference, let's include the value we saw from our experiment -- the green line.

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

ggplot() + 
    geom_histogram(aes(x=tabclick),newtrials,bins=30,color="white",fill="lightblue") +
    geom_vline(xintercept=1244,color="green")+
    xlim(900,1250)+
    ggtitle("Null Distribution")

Notice how nicely bell-shaped the null distribution is! We can use our new-found plot, the normal qqplot, to ssess it more formally.

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)

ggplot() +
    geom_qq(aes(sample=tabclick),newtrials)

What do you think?

This is the soul of significance testing. In later incarnations, competing camps tried to firm up significance testing by casting it as a decision problem. Hypothesis testing was developed by Jerzy Neyman and Egon Pearson and involves not only a null hypothesis but also an alternative. 

Neyman and Pearson talk specifically about errors you can make in this testing process (rejecting a true null hypothesis or failing to reject a false null hypothesis, say). By accounting for these two errors (called Type I and Type II), they provided tools for researchers to balance the kinds of mistakes they might make. But there's no new magic here - in fact Neyman and Pearson decided that these rules were important because in their estimation, you really couldn't learn anything from data. You could only behave in ways that protected you from making mistakes. It was a kind of behavioral induction.

Finally, a made-up data set corresponding to Ali's headline from the New York Times. The comparison was between

> Headline 1: Tax Bill Clears Senate Panel as Support Widens Among G.O.P.

> Headline 2: Republicans Who Wavered on Tax Bill Now Signal Support

The second headline won out 4-1. We created a data set like `nyt` with the right statistics.

In [ ]:
nyt_headline <- read.csv("https://github.com/cocteau/breakfasts/raw/master/data/nyt2.csv")

table(select(nyt_headline,Variation,IfClicked))

In [ ]:
newnyt <- nyt_headline

newnyt$Variation <- sample(nyt_headline$Variation)
newtrials <- summarize(newnyt,h2click=sum(Variation=="H2" & IfClicked==1))

for(i in 1:1000){
    newnyt$Variation <- sample(nyt_headline$Variation)
    newtrials <- bind_rows(newtrials,
                           summarize(newnyt,h2click=sum(Variation=="H2" & IfClicked==1)))
}

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

ggplot() + 
    geom_histogram(aes(x=h2click),newtrials,bins=30,color="white",fill="lightblue") +
    ggtitle("Null Distribution")

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

ggplot() + 
    geom_histogram(aes(x=h2click),newtrials,bins=100,color="white",fill="lightblue") +
    geom_vline(xintercept=4000,color="green")+
    xlim(2400,4000)+
    ggtitle("Null Distribution")

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)

ggplot() +
    geom_qq(aes(sample=h2click),newtrials)

This, of course, is just a taste of statistical reasoning. You've seen a couple frequentist devices for assessing uncertainty in a data set. Note how the deployment of randomness lets us reason about our survey results or about an experiment we conducted. 

If you are still interested, we can continue next term on modeling, from regression to decision trees. 